# Run this in Kaggle

🏙️ 1️⃣ Introduction

This project builds an AI-powered system capable of detecting and reporting
various civic infrastructure issues such as:

• Potholes and road cracks  
• Fallen trees  
• Garbage accumulation  
• Graffiti  
• Damaged road signs  
• Damaged electrical poles  
• Damaged concrete structures  

The goal is to train a YOLOv8 object detection model using the **Urban Issues Dataset**
and later deploy it inside a full web+mobile complaint reporting pipeline.

This notebook performs:

1. Dataset exploration  
2. Class-level image distribution  
3. Data cleaning (removing low-sample classes)  
4. YOLOv8 dataset preparation  
5. YOLO model training  
6. Evaluation and visualization  
7. Export of trained model for deployment


🗂️ 2️⃣ About the Dataset

Dataset: Urban Issues Dataset  
Link: https://www.kaggle.com/datasets/akinduhiman/urban-issues-dataset

The dataset contains images of real-world civic issues categorized into folders.
Each folder contains a YOLO-style structure:

<CLASS_NAME>/
    <CLASS_NAME>/
        train/
            images/
            labels/
        valid/
            images/
            labels/
        test/
            images/
            labels/

Each annotation uses YOLO format:
class_id  x_center  y_center  width  height

Total classes available:  
- Damaged concrete structures  
- DamagedElectricalPoles  
- DamagedRoadSigns  
- FallenTrees  
- Garbage  
- Graffitti  
- Potholes and RoadCracks  
- IllegalParking  
- DeadAnimalsPollution  

Two categories have very low samples and will be removed to avoid poor YOLO training:

Removed:
• IllegalParking  
• DeadAnimalsPollution  


📁 3️⃣ Load Dataset & Inspect Directory Structure

## Step 1 — Load dataset & display class folders

We begin by checking the dataset path and verifying all available class directories.


In [ ]:
# ============================
# Step 1: Basic Imports
# ============================

import os
from pathlib import Path
import numpy as np
import pandas as pd
from PIL import Image

# For checking directory structure
import os

# Raw dataset root (the input dataset on Kaggle)
DATASET_ROOT = Path("/kaggle/input/urban-issues-dataset")

print("Dataset root:", DATASET_ROOT)

# List all top-level directories (classes)
classes = sorted([d.name for d in DATASET_ROOT.iterdir() if d.is_dir()])
print("\nDetected class folders:")
for c in classes:
    print(" -", c)


📊 4️⃣ Count Images per Class & Split

## Step 2 — Count number of images in train/valid/test splits

Each class folder contains its own train/valid/test set.
This step summarizes the number of samples per split for each class.


In [ ]:
# ============================
# Step 2: Count images per split per class
# ============================

import os
from pathlib import Path

DATASET_ROOT = Path("/kaggle/input/urban-issues-dataset")

def count_images_in_split(class_folder):
    """
    Given class name, count number of images in train/valid/test/images.
    """
    base = DATASET_ROOT / class_folder / class_folder

    counts = {
        "train": 0,
        "valid": 0,
        "test": 0
    }

    # define split folders
    split_paths = {
        "train": base / "train" / "images",
        "valid": base / "valid" / "images",
        "test": base / "test" / "images"
    }

    for split, folder in split_paths.items():
        if folder.exists():
            counts[split] = len([f for f in folder.iterdir() if f.suffix.lower() in [".jpg", ".jpeg", ".png"]])
        else:
            counts[split] = 0

    return counts


# Run counts for each class
summary = {}

classes = sorted([d.name for d in DATASET_ROOT.iterdir() if d.is_dir()])

for cls in classes:
    summary[cls] = count_images_in_split(cls)

# Print summary
import pandas as pd
df = pd.DataFrame(summary).T
df["total"] = df.sum(axis=1)
df


🧹 5️⃣ Remove Low-Sample Classes

## Step 3 — Filter usable classes (remove extremely low sample categories)

Two classes contain very few samples and negatively affect YOLO performance.
Hence they are removed:

Removed:
- IllegalParking
- DeadAnimalsPollution

Final selected classes (7):
- Damaged concrete structures
- DamagedElectricalPoles
- DamagedRoadSigns
- FallenTrees
- Garbage
- Graffitti
- Potholes and RoadCracks


In [ ]:
# ============================
# Step 3: Select final categories for YOLOv8 training
# ============================

ALL_CLASSES = sorted([d.name for d in DATASET_ROOT.iterdir() if d.is_dir()])

# Remove classes with very low image count
REMOVE_CLASSES = ["IllegalParking", "DeadAnimalsPollution"]

FINAL_CLASSES = [c for c in ALL_CLASSES if c not in REMOVE_CLASSES]

print("Final classes to train on (7 classes):\n")
for c in FINAL_CLASSES:
    print(" -", c)

print("\nRemoved due to extremely low samples:\n")
for c in REMOVE_CLASSES:
    print(" -", c)


## Step 4 — Convert dataset into YOLOv8 format

YOLO requires a strict folder structure:

urban_yolo_dataset/
    images/
        train/
        val/
        test/
    labels/
        train/
        val/
        test/
    classes.txt

We will:
• Create the YOLO directory structure
• Copy images into train/val/test folders
• Remap class IDs for the 7 selected classes


In [ ]:
# ============================
# Step 4: Prepare YOLOv8 Dataset
# ============================

import shutil

WORK_ROOT = Path("/kaggle/working/urban_yolo_dataset")

# Re-create folders
if WORK_ROOT.exists():
    shutil.rmtree(WORK_ROOT)

# YOLO required structure
for split in ["train", "val", "test"]:
    (WORK_ROOT / "images" / split).mkdir(parents=True, exist_ok=True)
    (WORK_ROOT / "labels" / split).mkdir(parents=True, exist_ok=True)

print("YOLO workspace created at:", WORK_ROOT)

# -------- Class Mapping --------
FINAL_CLASSES = [
    "Damaged concrete structures",
    "DamagedElectricalPoles",
    "DamagedRoadSigns",
    "FallenTrees",
    "Garbage",
    "Graffitti",
    "Potholes and RoadCracks"
]

class2idx = {c: i for i, c in enumerate(FINAL_CLASSES)}
print("\nClass → Index mapping:")
print(class2idx)

# Save classes.txt
with open(WORK_ROOT / "classes.txt", "w") as f:
    f.write("\n".join(FINAL_CLASSES))


# -------- Copy Images & Labels --------

IMG_EXTS = (".jpg", ".jpeg", ".png", ".bmp")

counts = {"train": 0, "val": 0, "test": 0}

for cls in FINAL_CLASSES:
    cls_root = DATASET_ROOT / cls

    # each class folder contains: train/ valid/ test/
    for split_name in ["train", "valid", "test"]:
        split_path = cls_root / cls / split_name

        if not split_path.exists():
            continue

        image_dir = split_path / "images"
        label_dir = split_path / "labels"

        for img_name in os.listdir(image_dir):
            if not img_name.lower().endswith(IMG_EXTS):
                continue

            # source image
            src_img = image_dir / img_name

            # destination image
            new_name = f"{cls}__{img_name}"
            dst_img = WORK_ROOT / "images" / (
                "val" if split_name == "valid" else split_name
            ) / new_name

            shutil.copyfile(src_img, dst_img)

            # label file
            lbl_name = img_name.rsplit(".", 1)[0] + ".txt"
            src_lbl = label_dir / lbl_name

            dst_lbl = WORK_ROOT / "labels" / (
                "val" if split_name == "valid" else split_name
            ) / (new_name.rsplit(".", 1)[0] + ".txt")

            # remap class IDs
            if src_lbl.exists():
                lines = open(src_lbl).read().strip().split("\n")
                new_lines = []
                for line in lines:
                    parts = line.split()
                    if len(parts) < 5:
                        continue

                    # Replace class ID with our mapped 0–6
                    parts[0] = str(class2idx[cls])
                    new_lines.append(" ".join(parts))

                with open(dst_lbl, "w") as f:
                    f.write("\n".join(new_lines))
            else:
                open(dst_lbl, "w").close()

            # count
            tgt = "val" if split_name == "valid" else split_name
            counts[tgt] += 1

print("\nCopied image counts:", counts)
print("\nDataset preparation completed.")


## Step 5 — Validate YOLO dataset distribution

After restructuring the dataset, we count the total images per split and per class.
This ensures balanced datasets before training.


In [ ]:
# ============================
# Step 5 (fixed): Count samples inside YOLO workspace
# ============================

import os
from pathlib import Path
import pandas as pd

WORK_ROOT = Path("/kaggle/working/urban_yolo_dataset")

FINAL_CLASSES = [
    "Damaged concrete structures",
    "DamagedElectricalPoles",
    "DamagedRoadSigns",
    "FallenTrees",
    "Garbage",
    "Graffitti",
    "Potholes and RoadCracks"
]

class2idx = {c: i for i, c in enumerate(FINAL_CLASSES)}

# Prepare count table
counts = {
    cls: {"train": 0, "val": 0, "test": 0, "total": 0}
    for cls in FINAL_CLASSES
}

splits = ["train", "val", "test"]

for split in splits:
    label_dir = WORK_ROOT / "labels" / split

    for lbl_file in label_dir.iterdir():
        if lbl_file.suffix != ".txt":
            continue

        raw = open(lbl_file).read().strip()

        # skip empty label files
        if raw == "":
            continue

        lines = raw.split("\n")

        # First annotation line → class ID
        first_line = lines[0].split()
        if len(first_line) == 0:
            continue

        cls_id = int(first_line[0])
        cls_name = FINAL_CLASSES[cls_id]

        counts[cls_name][split] += 1
        counts[cls_name]["total"] += 1

# Convert to DataFrame
df_counts = pd.DataFrame(counts).T
df_counts


## Step 6 — Create data.yaml for YOLOv8 Training

YOLOv8 requires a YAML file that defines:
- dataset root path
- train/val/test folder locations
- class names in correct order


In [ ]:
# ============================
# Step 6: Create data.yaml
# ============================

import yaml
from pathlib import Path

yaml_path = Path("/kaggle/working/urban_yolo_dataset/data.yaml")

data_yaml = {
    "path": "/kaggle/working/urban_yolo_dataset",  # base dataset folder
    "train": "images/train",
    "val": "images/val",
    "test": "images/test",
    "names": [
        "Damaged concrete structures",
        "DamagedElectricalPoles",
        "DamagedRoadSigns",
        "FallenTrees",
        "Garbage",
        "Graffitti",
        "Potholes and RoadCracks"
    ]
}

with open(yaml_path, "w") as f:
    yaml.safe_dump(data_yaml, f)

print("data.yaml created at:", yaml_path)


## Step 9 — Train YOLOv8n model

We use YOLOv8n (nano) version for:
- Fast training
- Lower GPU usage
- Good accuracy for city surveillance datasets

Training settings:
- 30 epochs
- 640×640 resolution
- Early stopping (patience = 10)


In [ ]:
!pip install -q ultralytics

In [ ]:
from ultralytics import YOLO

print("Ultralytics version:", YOLO)


In [ ]:
# ============================
# Step 9: Train YOLOv8n
# ============================

from ultralytics import YOLO

# Load pretrained YOLOv8n model (fast, good for Kaggle)
model = YOLO("yolov8n.pt")

# Train
results = model.train(
    data="/kaggle/working/urban_yolo_dataset/data.yaml",  # your prepared dataset
    epochs=30,
    imgsz=640,
    batch=16,
    workers=2,
    patience=10,   # early stopping
    project="/kaggle/working/urban_yolo_dataset/training_runs",
    name="yolov8_urban_issues",
    pretrained=True
)

results


## Step 10 — Inspect training outputs

YOLO creates:
- last.pt
- best.pt (best model)
- results.png
- metrics.json


In [ ]:
# ============================
# Step 10: View training results
# ============================

import os
from pathlib import Path

RUNS_DIR = Path("/kaggle/working/urban_yolo_dataset/training_runs/yolov8_urban_issues")

print("Training run directory:")
print(RUNS_DIR)

print("\nAvailable files:")
for f in (RUNS_DIR / "weights").iterdir():
    print(" -", f.name)


## Step 11 — Evaluate Model on Validation/Test Sets

This step computes:
- mAP@50
- mAP@50-95
- Precision / Recall
- Confusion matrices


In [ ]:
# ============================
# Step 11: Evaluate YOLOv8 model
# ============================

from ultralytics import YOLO

best_model_path = "/kaggle/working/urban_yolo_dataset/training_runs/yolov8_urban_issues/weights/best.pt"

model = YOLO(best_model_path)

metrics = model.val()  # runs evaluation
metrics


## Step 13 — Save best model to output folder

This creates a clean /kaggle/working/best_model_yolov8.pt file,
used later in the Colab Web App.


In [ ]:
# ============================
# Step 13: Copy best model to Kaggle output
# ============================

import shutil

src = "/kaggle/working/urban_yolo_dataset/training_runs/yolov8_urban_issues/weights/best.pt"
dst = "/kaggle/working/best_model_yolov8.pt"

shutil.copyfile(src, dst)

print("Saved model to:", dst)


In [ ]:
# ============================
# Visualize Test Predictions (Random Samples)
# ============================

import os
import random
from ultralytics import YOLO
from IPython.display import display
from PIL import Image

# Load best model
model_path = "/kaggle/working/best_model_yolov8.pt"
model = YOLO(model_path)

# Path to test images
TEST_DIR = "/kaggle/working/urban_yolo_dataset/images/test"

# Pick 5 random test images
sample_images = random.sample(os.listdir(TEST_DIR), 5)

print("Showing predictions for:")
for img_name in sample_images:
    print(" -", img_name)

    img_path = f"{TEST_DIR}/{img_name}"

    # Run prediction
    results = model.predict(img_path, conf=0.25, imgsz=640)

    # Get annotated image
    annotated = results[0].plot()   # numpy array

    # Convert to PIL for display
    display(Image.fromarray(annotated))


# Run this in Colab

📘 1️⃣ Project Introduction — Colab Deployment Notebook

# AI Powered Civic Issue Reporting & Resolution System  
### Deployment Notebook (Flask Web App + YOLOv8 + LLM)

This notebook deploys the trained YOLOv8 model inside a complete AI-powered  
Civic Issue Reporting Web Application with:

• Automatic Image Detection (Potholes, Garbage, Fallen Trees, etc.)  
• AI-Generated Civic Issue Titles + Descriptions (LLM assisted)  
• Real-time Submission Dashboard  
• Admin Login System  
• Interactive Map Dashboard (Leaflet.js)  
• Public User Interface (Pending / Assigned / Resolved)  
• Ngrok-based public URL for mobile testing  

⚠️ **Model is not provided.**  
Users must train & export **best_model_yolov8.pt** using the Kaggle notebook.

This Colab notebook focuses on:

1. Mount Drive (upload trained model)  
2. Install dependencies  
3. Build Flask UI templates  
4. Load YOLO model from Drive  
5. Run Flask + expose using ngrok  
6. Test full civic reporting workflow  


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

📁 2️⃣ Loading Trained Model from Google Drive

## Step 1 — Mount Drive & confirm model exists

Your Kaggle notebook exported:
best_model_yolov8.pt

You must upload this file into:
MyDrive/Colab Notebooks/On-Going Projects/AI Powered Civic Issue Reporting & Resolution System - 10/

This cell:
• Mounts Google Drive  
• Verifies the `.pt` YOLO model file exists  


In [ ]:
!ls "/content/drive/MyDrive/Colab Notebooks/On-Going Projects/AI Powered Civic Issue Reporting & Resolution System - 10/best_model_yolov8.pt"

🛠️ 3️⃣ Install Required Packages

## Step 2 — Install all required packages

We install:
- Flask (web framework)
- pyngrok (public URL tunneling)
- ultralytics (YOLOv8)
- transformers + bitsandbytes (LLM for report generation)
- pillow (image handling)

Make sure GPU is enabled in runtime.


In [ ]:
!pip install -q flask pyngrok ultralytics transformers accelerate bitsandbytes pillow


## Step 3 — Create project folders

The Flask app needs:
templates/
static/
uploads/
db/

This step creates them if not already present.


In [ ]:
# Colab cell 2: create folders
!mkdir -p templates static uploads db


## Step 4 — Login to HuggingFace Hub

LLM used:
google/gemma-7b-it (8-bit quantized)

Enter your own token (never hardcode in production).


In [ ]:
# Colab cell 3: login to Hugging Face (replace token with your token)
from huggingface_hub import login
login(token="replace with your token")


## Step 5 — Ensure GPU is available

Used for:
• YOLOv8 inference  
• LLM 8-bit quantization

If `torch.cuda.is_available() == False`, LLM may fallback to CPU.


In [ ]:
# ===============================
# 4️⃣ Check GPU availability
# ===============================
import torch
print("GPU Available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU Name:", torch.cuda.get_device_name(0))

## Step 6 — Create Navbar Template

Shared navigation bar displayed on all pages.
Contains:
• Home
• Report Issue
• Pending / Assigned / Resolved (User View)
• Admin Dashboard + Login / Logout (Admin View)

System automatically hides/shows based on session state.


In [ ]:
%%writefile templates/navbar.html
<header class="topbar">
  <div class="topbar-left">
    <span class="logo-icon">🏙️</span>
    <div>
      <div class="brand-title">CivicCare</div>
      <div class="brand-sub">AI-Powered Civic Solutions</div>
    </div>
  </div>

  <div class="topbar-right">

    <!-- ALWAYS VISIBLE -->
    <a href="{{ url_for('home') }}"
       class="{{ 'active' if request.path in ['/', url_for('home')] else '' }}">
       Home
    </a>

    <!-- Only users should see Report Issue -->
    {% if not session.get('admin') %}
      <a href="{{ url_for('report') }}"
         class="{{ 'active' if request.path == url_for('report') else '' }}">
         Report Issue
      </a>
    {% endif %}

    <!-- ============================
         USER VIEW (read-only pages)
         ============================ -->
    {% if not session.get('admin') %}
      <a href="{{ url_for('user_pending') }}"
         class="{{ 'active' if request.path == url_for('user_pending') else '' }}">
         Pending
      </a>

      <a href="{{ url_for('user_assigned') }}"
         class="{{ 'active' if request.path == url_for('user_assigned') else '' }}">
         Assigned
      </a>

      <a href="{{ url_for('user_resolved') }}"
         class="{{ 'active' if request.path == url_for('user_resolved') else '' }}">
         Resolved
      </a>
    {% endif %}

    <!-- ============================
         ADMIN VIEW
         ============================ -->
    {% if session.get('admin') %}

      <a href="{{ url_for('admin') }}"
         class="{{ 'active' if request.path == url_for('admin') else '' }}">
         Dashboard
      </a>

      <a href="{{ url_for('admin_panel') }}"
         class="{{ 'active' if request.path == url_for('admin_panel') else '' }}">
         Pending
      </a>

      <a href="{{ url_for('admin_assigned') }}"
         class="{{ 'active' if request.path == url_for('admin_assigned') else '' }}">
         Assigned
      </a>

      <a href="{{ url_for('admin_resolved') }}"
         class="{{ 'active' if request.path == url_for('admin_resolved') else '' }}">
         Resolved
      </a>

      <a href="{{ url_for('admin_logout') }}">Logout</a>

    {% else %}
      <!-- Show Admin Login only when NOT logged in -->
      <a href="{{ url_for('admin_login') }}"
         class="{{ 'active' if request.path == url_for('admin_login') else '' }}">
         Admin Login
      </a>
    {% endif %}

  </div>
</header>



## Step 7 — Build Flask backend (app.py)

Modules included:
• YOLO model loader
• LLM loader (8-bit quantized Gemma)
• Civic Issue analyzer API (/api/analyze_image)
• Issue submission API (/api/submit_issue)
• Admin login system
• Admin control APIs (assign, resolve)
• Public pages (pending, assigned, resolved)
• Dynamic map dashboard (Leaflet)

Database:
db/issues.json
Contains all issue objects.


In [ ]:
%%writefile app.py
import os
import uuid
import json
from datetime import datetime, timedelta
from pathlib import Path

from flask import (
    Flask, render_template, request, jsonify,
    send_from_directory, redirect, url_for, session
)
from werkzeug.utils import secure_filename

import torch
from ultralytics import YOLO
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# -------------------------------------------
# SYSTEM CUDA FIXES
# -------------------------------------------
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
torch.cuda.empty_cache()

# -------------------------------------------
# FLASK CONFIG
# -------------------------------------------
app = Flask(__name__, template_folder="templates", static_folder="static")
app.secret_key = "super_secure_secret_123"

UPLOAD_DIR = "uploads"
DB_DIR = Path("db")
DB_PATH = DB_DIR / "issues.json"
ADMIN_PATH = DB_DIR / "admin.json"

os.makedirs(UPLOAD_DIR, exist_ok=True)
os.makedirs(DB_DIR, exist_ok=True)

if not DB_PATH.exists():
    DB_PATH.write_text("[]", encoding="utf-8")

if not ADMIN_PATH.exists():
    ADMIN_PATH.write_text(
        json.dumps({"username": "civicadmin", "password": "city123"}, indent=2),
        encoding="utf-8"
    )

# -------------------------------------------
# YOLO + CATEGORY CONFIG
# -------------------------------------------
YOLO_MODEL_PATH = "/content/drive/MyDrive/Colab Notebooks/On-Going Projects/AI Powered Civic Issue Reporting & Resolution System - 10/best_model_yolov8.pt"

CACHE = {"detector": None, "llm": None, "tokenizer": None}

YOLO_CLASSES = [
    "Damaged concrete structures",
    "DamagedElectricalPoles",
    "DamagedRoadSigns",
    "FallenTrees",
    "Garbage",
    "Graffitti",
    "Potholes and RoadCracks"
]

MODEL_TO_DISPLAY = {
    "Damaged concrete structures": "Damaged Concrete Structures",
    "DamagedElectricalPoles": "Damaged Electrical Poles",
    "DamagedRoadSigns": "Damaged Road Signs",
    "FallenTrees": "Fallen Trees",
    "Garbage": "Garbage",
    "Graffitti": "Graffiti",
    "Potholes and RoadCracks": "Potholes & Road Cracks"
}

DEFAULT_DISPLAY = "Other"

# -------------------------------------------
# LOAD ALL MODELS
# -------------------------------------------
def load_models():
    if CACHE["detector"] is None:
        try:
            CACHE["detector"] = YOLO(YOLO_MODEL_PATH)
        except Exception:
            CACHE["detector"] = None

    if CACHE["llm"] is None:
        # Lazy-load LLM only if available; keep resilient if offline
        try:
            model_name = "google/gemma-7b-it"
            bnb = BitsAndBytesConfig(load_in_8bit=True)

            CACHE["tokenizer"] = AutoTokenizer.from_pretrained(model_name, use_fast=True)
            CACHE["llm"] = AutoModelForCausalLM.from_pretrained(
                model_name,
                quantization_config=bnb,
                device_map="cuda",
                torch_dtype=torch.float16
            )
        except Exception:
            CACHE["llm"] = None
            CACHE["tokenizer"] = None

    return CACHE["detector"], CACHE["llm"], CACHE["tokenizer"]

# -------------------------------------------
# DB HELPERS
# -------------------------------------------
def read_db():
    try:
        return json.loads(DB_PATH.read_text(encoding="utf-8"))
    except:
        return []

def write_db(data):
    DB_PATH.write_text(json.dumps(data, indent=2), encoding="utf-8")

# -------------------------------------------
# LLM TEXT GENERATION (resilient)
# -------------------------------------------
def generate_text(category, labels, lat, lon):
    _, llm, tok = load_models()
    if llm is None or tok is None:
        return {
            "title": f"{category} issue reported",
            "description": f"Civic issue related to {category}."
        }

    prompt = f"""
    Write a JSON civic issue report.

    Category: {category}
    Detected Objects: {labels}
    Location: lat={lat}, lon={lon}

    Return JSON only.
    {{
        "title": "...",
        "description": "..."
    }}
    """

    inputs = tok(prompt, return_tensors="pt").to(llm.device)

    with torch.inference_mode():
        out = llm.generate(
            **inputs,
            max_new_tokens=150,
            temperature=0.6,
            do_sample=True,
            pad_token_id=tok.eos_token_id
        )

    text = tok.decode(out[0], skip_special_tokens=True)

    try:
        s = text.index("{")
        e = text.rindex("}") + 1
        return json.loads(text[s:e])
    except:
        return {
            "title": f"{category} issue reported",
            "description": f"Civic issue related to {category}."
        }

# -------------------------------------------
# API: ANALYZE IMAGE
# -------------------------------------------
@app.route("/api/analyze_image", methods=["POST"])
def analyze_image():
    try:
        if "image" not in request.files:
            return jsonify({"error": "No image uploaded"}), 400

        img = request.files["image"]
        name = f"{uuid.uuid4().hex}_{secure_filename(img.filename)}"
        path = os.path.join(UPLOAD_DIR, name)
        img.save(path)

        yolo, _, _ = load_models()
        detected = []
        best_class = None
        best_conf = 0.0

        if yolo is not None:
            results = yolo(path)
            for r in results:
                for box in r.boxes:
                    cls = int(box.cls[0])
                    raw = YOLO_CLASSES[cls] if cls < len(YOLO_CLASSES) else None
                    display = MODEL_TO_DISPLAY.get(raw, DEFAULT_DISPLAY)
                    conf = float(box.conf[0])

                    detected.append(display)

                    if conf > best_conf:
                        best_conf = conf
                        best_class = display

            try:
                det_path = os.path.join(UPLOAD_DIR, f"det_{name}")
                results[0].save(det_path)
                annotated_path = f"/uploads/det_{name}"
            except Exception:
                annotated_path = None
        else:
            # fallback if YOLO not available
            best_class = DEFAULT_DISPLAY
            annotated_path = None

        if not best_class:
            best_class = DEFAULT_DISPLAY

        lat = request.form.get("lat", "")
        lon = request.form.get("lon", "")

        ai = generate_text(best_class, detected, lat, lon)

        return jsonify({
            "success": True,
            "file": f"/uploads/{name}",
            "annotated": annotated_path,
            "category": best_class,
            "labels": list(dict.fromkeys(detected)),
            "confidence": best_conf,
            "ai_title": ai["title"],
            "ai_description": ai["description"]
        })

    except Exception as e:
        return jsonify({"error": str(e)}), 500

# -------------------------------------------
# API: SUBMIT ISSUE
# -------------------------------------------
@app.route("/api/submit_issue", methods=["POST"])
def submit_issue():
    try:
        data = request.json
        now = datetime.utcnow() + timedelta(hours=5, minutes=30)

        raw = data.get("category", "")
        category = MODEL_TO_DISPLAY.get(raw, raw) or DEFAULT_DISPLAY

        issue = {
            "id": uuid.uuid4().hex,
            "title": data.get("title", "") or f"{category} issue reported",
            "description": data.get("description", "") or f"Civic issue related to {category}.",
            "category": category,
            "lat": data.get("lat", ""),
            "lon": data.get("lon", ""),
            "image": data.get("image"),
            "annotated": data.get("annotated"),
            "created_at": now.strftime("%Y-%m-%d %H:%M:%S"),
            "status": "pending"
        }

        db = read_db()
        db.append(issue)
        write_db(db)

        return jsonify({"status": "ok", "id": issue["id"]})

    except Exception as e:
        return jsonify({"error": str(e)}), 500

# -------------------------------------------
# ADMIN LOGIN / LOGOUT
# -------------------------------------------
@app.route("/admin_login", methods=["GET", "POST"])
def admin_login():
    if request.method == "GET":
        return render_template("admin_login.html")

    u = request.form.get("username")
    p = request.form.get("password")

    admin = json.loads(ADMIN_PATH.read_text(encoding="utf-8"))

    if u == admin["username"] and p == admin["password"]:
        session["admin"] = True
        # redirect admin to map dashboard
        return redirect(url_for("admin"))
    return render_template("admin_login.html", error="Invalid credentials")

@app.route("/admin_logout")
def admin_logout():
    session.pop("admin", None)
    return redirect(url_for("home"))

# -------------------------------------------
# ADMIN PAGES (protected)
# -------------------------------------------
def require_admin():
    if "admin" not in session:
        return False
    return True

@app.route("/admin")
def admin():
    if not require_admin():
        return redirect(url_for("admin_login"))
    # admin map dashboard (same /admin as before)
    return render_template("admin.html")

@app.route("/admin_panel")
def admin_panel():
    if not require_admin():
        return redirect(url_for("admin_login"))
    db = read_db()
    pending = [i for i in db if i.get("status") == "pending"]
    return render_template("admin_panel.html", pending=pending)

@app.route("/admin_assigned")
def admin_assigned():
    if not require_admin():
        return redirect(url_for("admin_login"))
    db = read_db()
    assigned = [i for i in db if i.get("status") == "assigned"]
    return render_template("admin_assigned.html", assigned=assigned)

@app.route("/admin_resolved")
def admin_resolved():
    if not require_admin():
        return redirect(url_for("admin_login"))
    db = read_db()
    resolved = [i for i in db if i.get("status") == "resolved"]
    return render_template("admin_resolved.html", resolved=resolved)

# -------------------------------------------
# API: ADMIN ACTIONS (protected)
# -------------------------------------------
@app.route("/api/admin_assign_issue", methods=["POST"])
def admin_assign_api():
    if not require_admin():
        return jsonify({"error": "Unauthorized"}), 401

    issue_id = request.json.get("id")
    db = read_db()
    for i in db:
        if i["id"] == issue_id:
            i["status"] = "assigned"
            i["assigned_at"] = (datetime.utcnow() + timedelta(hours=5, minutes=30)).strftime("%Y-%m-%d %H:%M:%S")
            break
    write_db(db)
    return jsonify({"status": "ok"})

@app.route("/api/admin_resolve_issue", methods=["POST"])
def admin_resolve_api():
    if not require_admin():
        return jsonify({"error": "Unauthorized"}), 401

    issue_id = request.json.get("id")
    db = read_db()
    for i in db:
        if i["id"] == issue_id:
            i["status"] = "resolved"
            i["resolved_at"] = (datetime.utcnow() + timedelta(hours=5, minutes=30)).strftime("%Y-%m-%d %H:%M:%S")
            break
    write_db(db)
    return jsonify({"status": "ok"})

# -------------------------------------------
# PUBLIC (USER) PAGES - read only lists
# -------------------------------------------
@app.route("/")
def home():
    return render_template("index.html")

@app.route("/report")
def report():
    # admin should not use report page in normal flow
    if session.get("admin"):
        return redirect(url_for("admin"))
    return render_template("report.html")

@app.route("/pending")
def pending_list():
    db = read_db()
    pending = [i for i in db if i.get("status") == "pending"]
    return render_template("public_list.html", issues=pending, title="Pending Issues")

@app.route("/assigned")
def assigned_list():
    db = read_db()
    assigned = [i for i in db if i.get("status") == "assigned"]
    return render_template("public_list.html", issues=assigned, title="Assigned Issues")

@app.route("/resolved")
def resolved_list():
    db = read_db()
    resolved = [i for i in db if i.get("status") == "resolved"]
    return render_template("public_list.html", issues=resolved, title="Resolved Issues")

@app.route("/issue/<issue_id>")
def issue_page(issue_id):
    db = read_db()
    for item in db:
        if item["id"] == issue_id:
            return render_template("issue.html", issue=item)
    return "Issue Not Found", 404

@app.route("/uploads/<path:filename>")
def uploads(filename):
    return send_from_directory(UPLOAD_DIR, filename)

@app.route("/db/<path:filename>")
def serve_db(filename):
    return send_from_directory(str(DB_DIR), filename)

# Add these routes to your app.py (near other route definitions)
@app.route("/user_pending")
def user_pending():
    # public, read-only view of pending issues
    db = read_db()
    pending = [i for i in db if i.get("status") == "pending"]
    # render a read-only page — no admin buttons
    return render_template("user_pending.html", pending=pending)

@app.route("/user_assigned")
def user_assigned():
    db = read_db()
    assigned = [i for i in db if i.get("status") == "assigned"]
    return render_template("user_assigned.html", assigned=assigned)

@app.route("/user_resolved")
def user_resolved():
    db = read_db()
    resolved = [i for i in db if i.get("status") == "resolved"]
    return render_template("user_resolved.html", resolved=resolved)


# -------------------------------------------
# RUN APP
# -------------------------------------------
if __name__ == "__main__":
    # try to pre-load models but continue gracefully if not possible
    try:
        load_models()
    except Exception:
        pass
    app.run(host="0.0.0.0", port=8000)




## Step 8 — Build Home Page

Features:
• Hero section
• CTA buttons
• Overview of system capabilities
• AI + Civic engagement design


In [ ]:
%%writefile templates/index.html
<!doctype html>
<html lang="en">
<head>
  <meta charset="utf-8" />
  <meta name="viewport" content="width=device-width,initial-scale=1" />
  <title>CivicCare — AI-Powered Civic Issue Reporting</title>
  <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">
</head>
<body class="{% if session.get('admin') %}admin{% else %}user{% endif %}">

<!-- Shared Navbar -->
{% include 'navbar.html' %}

<div class="particles"></div>

<main class="container">
    <section class="hero-section">
      <div class="hero-left">
        <h1 class="main-title">CivicCare</h1>
        <div class="hero-subtitle">Smart Civic Issue Reporting System</div>

        <p class="hero-description">
          Report civic issues with AI assistance. Upload a photo, and our advanced
          AI models automatically detect the problem type, generate professional
          descriptions, and help municipal authorities respond faster. Making cities
          smarter, safer, and more responsive.
        </p>

        <div class="cta-row">
          <a class="cta-button" href="{{ url_for('report') }}">
            📸 Report Issue Now
          </a>

          {% if session.get('admin') %}
          <a class="cta-button-secondary" href="{{ url_for('admin') }}">
            🗺️ Admin Dashboard
          </a>
          {% else %}
          <a class="cta-button-secondary" href="{{ url_for('admin_login') }}">
            🔐 Admin Login
          </a>
          {% endif %}
        </div>

        <div class="stats-section">
          <div class="stat-card">
            <div class="stat-number">AI-Powered</div>
            <div class="stat-label">Auto Detection</div>
          </div>
          <div class="stat-card">
            <div class="stat-number">Real-Time</div>
            <div class="stat-label">Tracking</div>
          </div>
          <div class="stat-card">
            <div class="stat-number">Secure</div>
            <div class="stat-label">Platform</div>
          </div>
        </div>
      </div>

      <div class="hero-right">
        <div class="floating-card">
          <div class="card-icon-large">🤖</div>
          <h3>AI Detection</h3>
          <p>Advanced models identify potholes, garbage, streetlights, and more</p>
        </div>
      </div>
    </section>

    <section class="features-grid">
      <div class="feature-card">
        <div class="card-icon">📷</div>
        <h3>Image Recognition</h3>
        <p>AI analyzes uploaded photos to automatically detect civic issues.</p>
      </div>

      <div class="feature-card">
        <div class="card-icon">🧠</div>
        <h3>Smart Categorization</h3>
        <p>Deep learning models classify issues with high accuracy.</p>
      </div>

      <div class="feature-card">
        <div class="card-icon">✍️</div>
        <h3>Auto-Generated Reports</h3>
        <p>LLM creates professional titles and descriptions for submissions.</p>
      </div>

      <div class="feature-card">
        <div class="card-icon">🗺️</div>
        <h3>Geolocation Mapping</h3>
        <p>All reports are mapped with accurate coordinates.</p>
      </div>

      <div class="feature-card">
        <div class="card-icon">⚡</div>
        <h3>Real-Time Updates</h3>
        <p>Admins track issues as they move from pending to resolved.</p>
      </div>

      <div class="feature-card">
        <div class="card-icon">🔒</div>
        <h3>Secure & Private</h3>
        <p>End-to-end protection ensures user data safety.</p>
      </div>
    </section>

    <footer class="footer">
      <div>
        <strong>CivicCare</strong> — Empowering citizens and municipal teams with AI.
      </div>
      <div style="margin-top:8px;font-size:13px;opacity:0.7">
        Powered by Transformers & YOLO • Scalable • Secure
      </div>
    </footer>

</main>

</body>
</html>





## Step 9 — Build Issue Reporting UI

This page allows users to:
• Upload an image
• Auto-detect location
• Run AI analysis
• Preview YOLO detections
• Autofill AI-generated title + description
• Submit issue

JavaScript handles:
• Image preview  
• GPS detection  
• Analyze → /api/analyze_image  
• Submit → /api/submit_issue  


In [ ]:
%%writefile templates/report.html
<!doctype html>
<html lang="en">
<head>
  <meta charset="utf-8"/>
  <meta name="viewport" content="width=device-width,initial-scale=1"/>
  <title>Report Issue — CivicCare</title>
  <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">
</head>
<body class="user">

<!-- Shared Navbar -->
{% include 'navbar.html' %}

<div class="particles"></div>

<main class="container">
  <div class="page-header">
    <h2 class="page-title">Report a Civic Issue</h2>
    <p class="page-subtitle">Upload a photo and location. AI will analyze and assist your report.</p>
  </div>

  <div class="main-form">

    <!-- Step 1 -->
    <div class="form-section">
      <h3 class="section-title">📸 Step 1: Upload Issue Photo</h3>
      <p class="section-desc">Choose or capture a clear image of the issue</p>

      <div class="upload-area" id="uploadArea">
        <input id="image" type="file" accept="image/*" style="display:none" onchange="handleImageSelect(event)">
        <div class="upload-placeholder" id="uploadPlaceholder" onclick="document.getElementById('image').click()">
          <div class="upload-icon">📷</div>
          <div>Click to upload or drag & drop</div>
          <div style="font-size:13px;opacity:0.6;margin-top:4px">JPG, PNG, HEIC (Max 10MB)</div>
        </div>
        <img id="imagePreview" class="image-preview" style="display:none" alt="Original preview">
      </div>

      <div id="fileNameDisplay" style="display:none; margin-top:12px; padding:10px; background:rgba(16,185,129,0.1); border:1px solid rgba(16,185,129,0.3); border-radius:8px; color:#6ee7b7; font-size:14px;">
        <strong>✅ Selected:</strong> <span id="fileName"></span>
      </div>

      <div class="form-row">
        <div class="form-col">
          <label class="input-label">📍 Latitude</label>
          <input id="lat" type="text" placeholder="e.g., 17.3850">
          <button class="detect-btn" onclick="detectLocation()">📍 Auto-Detect</button>
        </div>

        <div class="form-col">
          <label class="input-label">📍 Longitude</label>
          <input id="lon" type="text" placeholder="e.g., 78.4867">
        </div>
      </div>

      <div class="action-row">
        <button class="submit-button primary" id="analyzeBtn" onclick="analyzeImage()">
          🤖 Analyze with AI
        </button>
        <button class="submit-button ghost" onclick="clearFields()">
          🗑️ Clear All
        </button>
      </div>
    </div>

    <!-- Step 2 -->
    <div id="aiSection" class="form-section" style="display:none">
      <h3 class="section-title">🤖 Step 2: AI Analysis Results</h3>
      <p class="section-desc">AI filled these automatically — edit if needed</p>

      <div class="ai-badge" id="categoryBadge"></div>

      <label class="input-label">Detected Category</label>
      <input id="category" type="text" readonly>

      <label class="input-label">AI-Generated Title (Editable)</label>
      <input id="title" type="text">

      <label class="input-label">AI-Generated Description (Editable)</label>
      <textarea id="description" rows="5"></textarea>

      <div class="confidence-display" id="confidenceDisplay"></div>

      <div style="display:flex;gap:12px;margin-top:12px;flex-wrap:wrap">
        <div style="flex:1;min-width:200px">
          <div style="font-size:13px;color:var(--muted);margin-bottom:6px">Original Upload</div>
          <img id="aiOriginal" src="" style="width:100%;border-radius:8px;display:none;object-fit:cover;max-height:180px">
        </div>

        <div style="flex:1;min-width:200px">
          <div style="font-size:13px;color:var(--muted);margin-bottom:6px">AI Annotated</div>
          <img id="aiAnnotated" src="" style="width:100%;border-radius:8px;display:none;object-fit:cover;max-height:180px">
        </div>
      </div>

      <div class="action-row" style="margin-top:16px">
        <button class="submit-button success" id="submitBtn" onclick="submitIssue()">
          ✅ Submit Issue Report
        </button>

        {% if session.get('admin') %}
        <a class="view-link" href="{{ url_for('admin') }}">View Dashboard →</a>
        {% else %}
        <a class="view-link" href="{{ url_for('admin_login') }}">Admin Login →</a>
        {% endif %}
      </div>
    </div>

    <div id="status" class="status-message"></div>

  </div>
</main>

<script>
let uploadedFile = null;
window.UPLOAD_PATH = null;
window.ANNOTATED_PATH = null;

function handleImageSelect(event) {
  const file = event.target.files[0];
  if (!file) return;

  uploadedFile = file;

  document.getElementById('fileName').textContent = file.name;
  document.getElementById('fileNameDisplay').style.display = 'block';

  const reader = new FileReader();
  reader.onload = (e) => {
    const img = document.getElementById('imagePreview');
    img.src = e.target.result;
    img.style.display = 'block';
    document.getElementById('uploadPlaceholder').style.display = 'none';
  };
  reader.readAsDataURL(file);

  setStatus("✅ Image uploaded successfully! Now add location and click 'Analyze with AI'.", "success");
}

function detectLocation() {
  if (navigator.geolocation) {
    setStatus("🔍 Detecting your location...", "info");
    navigator.geolocation.getCurrentPosition(
      (position) => {
        document.getElementById("lat").value = position.coords.latitude.toFixed(6);
        document.getElementById("lon").value = position.coords.longitude.toFixed(6);
        setStatus("✅ Location detected successfully!", "success");
      },
      () => setStatus("❌ Could not detect location. Please enter manually.", "error")
    );
  }
}

async function analyzeImage() {
  const file = uploadedFile || document.getElementById("image").files[0];
  const lat = document.getElementById("lat").value.trim();
  const lon = document.getElementById("lon").value.trim();

  if (!file) return setStatus("❌ Please select an image first", "error");
  if (!lat || !lon) return setStatus("⚠️ Please provide location coordinates", "warning");

  const fd = new FormData();
  fd.append("image", file);
  fd.append("lat", lat);
  fd.append("lon", lon);

  setStatus("🤖 AI is analyzing your image...", "info");
  const analyzeBtn = document.getElementById('analyzeBtn');
  analyzeBtn.disabled = true;
  analyzeBtn.textContent = "⏳ Analyzing...";

  try {
    const r = await fetch("/api/analyze_image", { method: "POST", body: fd });
    const j = await r.json();

    if (j.error) {
      setStatus("❌ Analysis failed: " + j.error, "error");
      analyzeBtn.disabled = false;
      analyzeBtn.textContent = "🤖 Analyze with AI";
      return;
    }

    window.UPLOAD_PATH = j.file || null;
    window.ANNOTATED_PATH = j.annotated || null;

    document.getElementById("aiSection").style.display = "block";
    document.getElementById("aiSection").scrollIntoView({ behavior: 'smooth' });

    document.getElementById("category").value = j.category || "other";
    document.getElementById("title").value = j.ai_title || "";
    document.getElementById("description").value = j.ai_description || "";

    if (window.UPLOAD_PATH) {
      const o = document.getElementById("aiOriginal");
      o.src = window.UPLOAD_PATH;
      o.style.display = "block";
    }

    if (window.ANNOTATED_PATH) {
      const a = document.getElementById("aiAnnotated");
      a.src = window.ANNOTATED_PATH;
      a.style.display = "block";
    }

    const badge = document.getElementById("categoryBadge");
    const cat = j.category ? j.category.replace(/ /g, '-').toLowerCase() : "other";
    badge.textContent = `Detected: ${j.category ? j.category.toUpperCase() : 'OTHER'}`;
    badge.className = `ai-badge ${cat}`;

    if (j.labels && j.labels.length > 0) {
      const conf = document.getElementById("confidenceDisplay");
      conf.innerHTML = `<strong>🎯 Detected Labels:</strong> ${j.labels.join(", ")}`;
      conf.style.display = "block";
    }

    setStatus("✅ AI analysis complete!", "success");

  } catch (e) {
    setStatus("❌ Network error: " + e.message, "error");
  } finally {
    analyzeBtn.disabled = false;
    analyzeBtn.textContent = "🤖 Analyze with AI";
  }
}

async function submitIssue() {
  const payload = {
    title: document.getElementById("title").value.trim(),
    description: document.getElementById("description").value.trim(),
    lat: document.getElementById("lat").value.trim(),
    lon: document.getElementById("lon").value.trim(),
    category: document.getElementById("category").value.trim(),
    image: window.UPLOAD_PATH || null,
    annotated: window.ANNOTATED_PATH || null
  };

  if (!payload.title || !payload.description)
    return setStatus("❌ Please provide both title and description", "error");

  if (!payload.lat || !payload.lon)
    return setStatus("❌ Location coordinates are required", "error");

  setStatus("📤 Submitting your report...", "info");
  const submitBtn = document.getElementById('submitBtn');
  submitBtn.disabled = true;
  submitBtn.textContent = "⏳ Submitting...";

  try {
    const r = await fetch("/api/submit_issue", {
      method: "POST",
      headers: {"Content-Type": "application/json"},
      body: JSON.stringify(payload)
    });
    const j = await r.json();

    if (j.status === "ok") {
      setStatus("✅ Issue submitted successfully! Redirecting...", "success");
      setTimeout(() => (window.location.href = "/admin"), 1200);
    } else {
      setStatus("❌ Submission failed: " + (j.error || "Unknown error"), "error");
      submitBtn.disabled = false;
      submitBtn.textContent = "✅ Submit Issue Report";
    }
  } catch (e) {
    setStatus("❌ Network error: " + e.message, "error");
    submitBtn.disabled = false;
    submitBtn.textContent = "✅ Submit Issue Report";
  }
}

function clearFields() {
  document.getElementById("image").value = "";
  document.getElementById("lat").value = "";
  document.getElementById("lon").value = "";
  document.getElementById("category").value = "";
  document.getElementById("title").value = "";
  document.getElementById("description").value = "";
  document.getElementById("imagePreview").style.display = "none";
  document.getElementById("aiOriginal").style.display = "none";
  document.getElementById("aiAnnotated").style.display = "none";
  document.getElementById("uploadPlaceholder").style.display = "flex";
  document.getElementById("aiSection").style.display = "none";
  document.getElementById("confidenceDisplay").style.display = "none";
  document.getElementById("fileNameDisplay").style.display = "none";
  window.UPLOAD_PATH = null;
  window.ANNOTATED_PATH = null;
  uploadedFile = null;
  setStatus("", "");
}

function setStatus(msg, type) {
  const el = document.getElementById("status");
  el.textContent = msg;
  el.className = `status-message ${type}`;
  el.style.display = msg ? "block" : "none";
}

const uploadArea = document.getElementById("uploadArea");
uploadArea.addEventListener("dragover", (e) => {
  e.preventDefault();
  uploadArea.style.borderColor = "var(--primary)";
});

uploadArea.addEventListener("dragleave", () => {
  uploadArea.style.borderColor = "rgba(255,255,255,0.1)";
});

uploadArea.addEventListener("drop", (e) => {
  e.preventDefault();
  uploadArea.style.borderColor = "rgba(255,255,255,0.1)";
  const files = e.dataTransfer.files;
  if (files.length > 0) {
    const file = files[0];
    if (file.type.startsWith("image/")) {
      document.getElementById("image").files = files;
      handleImageSelect({target: {files: [file]}});
    } else {
      setStatus("❌ Please upload an image file", "error");
    }
  }
});
</script>
</body>
</html>


## Step 10 — Leaflet.js Live Map Dashboard

Admins see:
• All issues plotted by coordinates  
• Category-based color markers  
• Filters (Potholes / Garbage / Graffiti / etc.)  
• Real-time data refresh (every 30 seconds)

This page is central to municipal monitoring.


In [ ]:
%%writefile templates/admin.html
<!doctype html>
<html lang="en">
<head>
  <meta charset="utf-8"/>
  <meta name="viewport" content="width=device-width,initial-scale=1"/>
  <title>CivicCare Admin Dashboard</title>

  <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">
  <link rel="stylesheet" href="https://unpkg.com/leaflet@1.9.3/dist/leaflet.css"/>
  <script src="https://unpkg.com/leaflet@1.9.3/dist/leaflet.js"></script>

  <style>
    .filter-btn.active {
      background: var(--primary);
      color: white;
    }
  </style>
</head>

<body class="admin">

<!-- SHARED NAVBAR -->
{% include 'navbar.html' %}

<main class="container">

  <div class="page-header">
    <h2 class="page-title">🗺️ Municipal Issue Map</h2>
    <p class="page-subtitle">Track issues across the city in real-time</p>
  </div>

  <div class="stats-row">
    <div class="stat-box">
      <div class="stat-icon">📊</div>
      <div class="stat-value" id="totalIssues">0</div>
      <div class="stat-label">Total Issues</div>
    </div>

    <div class="stat-box">
      <div class="stat-icon">⏳</div>
      <div class="stat-value" id="pendingIssues">0</div>
      <div class="stat-label">Pending</div>
    </div>

    <div class="stat-box">
      <div class="stat-icon">🔥</div>
      <div class="stat-value" id="todayIssues">0</div>
      <div class="stat-label">Today</div>
    </div>
  </div>

  <div class="filter-bar">

    <button class="filter-btn active" onclick="applyFilter('all', event)">
      All Issues
    </button>

    <button class="filter-btn" onclick="applyFilter('damagedconcretestructures', event)">🏗️ Damaged Concrete</button>
    <button class="filter-btn" onclick="applyFilter('damagedelectricalpoles', event)">⚡ Electrical Poles</button>
    <button class="filter-btn" onclick="applyFilter('damagedroadsigns', event)">🚧 Road Signs</button>
    <button class="filter-btn" onclick="applyFilter('fallentrees', event)">🌳 Fallen Trees</button>
    <button class="filter-btn" onclick="applyFilter('garbage', event)">🗑️ Garbage</button>
    <button class="filter-btn" onclick="applyFilter('graffiti', event)">🎨 Graffiti</button>
    <button class="filter-btn" onclick="applyFilter('potholesandroadcracks', event)">🕳️ Potholes</button>
    <button class="filter-btn" onclick="applyFilter('other', event)">📋 Other</button>

  </div>

  <div id="map" class="map-container"></div>

  <footer class="footer">
    <div>
      <strong>CivicCare Admin Panel</strong> — Tracking civic issues with AI
    </div>
    <div style="margin-top:6px;font-size:12px;opacity:0.6">
      Powered by Python + Flask + Leaflet Maps
    </div>
  </footer>

</main>

<script>
let map = L.map('map').setView([20.5937, 78.9629], 5);

L.tileLayer('https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', {
  maxZoom: 19,
  attribution: '© OpenStreetMap contributors'
}).addTo(map);

let markers = [];
let allIssues = [];

const categoryConfig = {
  'damagedconcretestructures': { icon: '🏗️', color: '#9ca3af' },
  'damagedelectricalpoles': { icon: '⚡', color: '#facc15' },
  'damagedroadsigns': { icon: '🚧', color: '#fb923c' },
  'fallentrees': { icon: '🌳', color: '#22c55e' },
  'garbage': { icon: '🗑️', color: '#ef4444' },
  'graffiti': { icon: '🎨', color: '#a855f7' },
  'potholesandroadcracks': { icon: '🕳️', color: '#6366f1' },
  'other': { icon: '📋', color: '#6b7280' }
};

function normalize(str) {
  return (str || "").toLowerCase().replace(/[^a-z0-9]/g, "");
}

async function loadIssues() {
  try {
    const r = await fetch("/db/issues.json");
    const issues = await r.json();

    allIssues = issues;
    updateStats(issues);
    renderMarkers(issues);

  } catch (e) {
    console.error("Error loading issues:", e);
  }
}

function updateStats(issues) {
  document.getElementById("totalIssues").textContent = issues.length;

  const pending = issues.filter(i => i.status === "pending").length;
  document.getElementById("pendingIssues").textContent = pending;

  const today = new Date().toISOString().split("T")[0];
  const todayCount = issues.filter(i => i.created_at?.startsWith(today)).length;
  document.getElementById("todayIssues").textContent = todayCount;
}

function renderMarkers(issues) {
  markers.forEach(m => map.removeLayer(m));
  markers = [];

  issues.forEach(issue => {
    const lat = parseFloat(issue.lat);
    const lon = parseFloat(issue.lon);
    if (isNaN(lat) || isNaN(lon)) return;

    const key = normalize(issue.category);
    const config = categoryConfig[key] || categoryConfig.other;

    const marker = L.marker([lat, lon], {
      icon: L.divIcon({
        html: `
          <div style="
            background:${config.color};
            width:32px;
            height:32px;
            border-radius:50%;
            display:flex;
            justify-content:center;
            align-items:center;
            border:3px solid white;
            font-size:18px;">
            ${config.icon}
          </div>`,
        className: "custom-marker",
        iconSize: [32, 32],
        iconAnchor: [16, 16],
      })
    }).addTo(map);

    marker.bindPopup(`
      <div class="popup-wrap">
        <h3>${issue.title}</h3>
        <div class="popup-category" style="background:${config.color}">
          ${config.icon} ${issue.category}
        </div>
        <p>${issue.description}</p>
        <div><strong>Status:</strong> ${issue.status}</div>
        <a href="/issue/${issue.id}" target="_blank" class="view-link">📄 View Details →</a>
      </div>
    `);

    markers.push(marker);
  });

  if (markers.length > 0) {
    map.fitBounds(L.featureGroup(markers).getBounds().pad(0.2));
  }
}

function applyFilter(cat, evt) {
  if (evt) {
    document.querySelectorAll(".filter-btn")
      .forEach(b => b.classList.remove("active"));
    evt.currentTarget.classList.add("active");
  }

  if (cat === "all") {
    renderMarkers(allIssues);
    updateStats(allIssues);
    return;
  }

  const filtered = allIssues.filter(i =>
    normalize(i.category).includes(cat)
  );

  renderMarkers(filtered);
  updateStats(filtered);
}

loadIssues();
setInterval(loadIssues, 30000);
</script>

</body>
</html>




## Step 11 — Admin Login Page

Default credentials stored in:
db/admin.json

Admins can:
• Access dashboard  
• Assign issues  
• Resolve issues  


## Step 12 — Build admin management pages

- admin_panel.html → Pending issues  
- admin_assigned.html → Assigned issues  
- admin_resolved.html → Resolved issues  

Admins can:
• Take up an issue  
• Mark resolved  


In [ ]:
%%writefile templates/admin_login.html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>CivicCare – Admin Login</title>

    <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">

    <style>
        .login-container {
            width: 420px;
            margin: 120px auto;
            padding: 30px;
            background: var(--glass);
            backdrop-filter: blur(12px);
            border-radius: 15px;
            box-shadow: 0 0 25px rgba(0,0,0,0.4);
            animation: fadeIn 0.6s ease;
        }

        .login-container h2 {
            margin-bottom: 18px;
            font-size: 26px;
            text-align: center;
        }

        .input-field {
            margin-bottom: 16px;
        }

        .input-field label {
            display: block;
            margin-bottom: 6px;
            font-size: 15px;
            color: var(--muted);
        }

        .input-field input {
            width: 100%;
            padding: 12px;
            border-radius: 8px;
            border: 1px solid #2b2f40;
            background: rgba(255,255,255,0.05);
            color: white;
            font-size: 15px;
            outline: none;
        }

        .btn-login {
            width: 100%;
            padding: 13px;
            background: var(--primary);
            color: white;
            font-size: 17px;
            border-radius: 8px;
            border: none;
            cursor: pointer;
            margin-top: 10px;
            transition: 0.3s;
        }

        .btn-login:hover {
            background: #4f4dec;
        }

        .error-msg {
            margin-top: 10px;
            text-align: center;
            color: #ff6b6b;
            font-size: 15px;
        }

        .back-home {
            text-align: center;
            margin-top: 14px;
        }

        .back-home a {
            color: var(--muted);
            text-decoration: none;
            font-size: 14px;
        }

        @keyframes fadeIn {
            from { opacity: 0; transform: translateY(20px); }
            to   { opacity: 1; transform: translateY(0); }
        }
    </style>

</head>
<body class="user">

<!-- SHARED NAVBAR -->
{% include 'navbar.html' %}

<div class="login-container">
    <h2>Admin Login</h2>

    {% if error %}
        <div class="error-msg">{{ error }}</div>
    {% endif %}

    <form method="POST" action="/admin_login">
        <div class="input-field">
            <label>Username</label>
            <input type="text" name="username" required placeholder="Enter admin username">
        </div>

        <div class="input-field">
            <label>Password</label>
            <input type="password" name="password" required placeholder="Enter admin password">
        </div>

        <button type="submit" class="btn-login">Login</button>
    </form>

    <div class="back-home">
        <a href="/">← Back to Home</a>
    </div>
</div>

</body>
</html>


In [ ]:
%%writefile templates/admin_panel.html
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8">
  <meta name="viewport" content="width=device-width, initial-scale=1.0">
  <title>CivicCare — Pending Issues</title>
  <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">
</head>

<body class="admin">

{% include "navbar.html" %}

<main class="container">

  <div class="panel-container">

    <div class="panel-header">
      <div class="panel-title">Pending Issues</div>
    </div>

    {% if pending|length == 0 %}
      <div class="no-issues">No pending issues.</div>
    {% endif %}

    {% for issue in pending %}
    <div class="issue-card">

      <div class="issue-title">{{ issue.title }}</div>

      <div class="issue-meta">
        Category: {{ issue.category }} |
        ID: {{ issue.id }} |
        Reported: {{ issue.created_at }}
      </div>

      <div class="issue-desc">{{ issue.description }}</div>

      <div class="action-buttons">
        <button class="btn-action btn-takeup"
                onclick="updateStatus('{{ issue.id }}','assigned')">🚀 Take Up Issue</button>

        <button class="btn-action btn-resolve"
                onclick="updateStatus('{{ issue.id }}','resolved')">✅ Mark Resolved</button>
      </div>

    </div>
    {% endfor %}

  </div>
</main>

<script>
async function updateStatus(id, state) {
  const url = state === "assigned"
    ? "/api/admin_assign_issue"
    : "/api/admin_resolve_issue";

  const r = await fetch(url, {
    method: "POST",
    headers: {"Content-Type": "application/json"},
    body: JSON.stringify({id})
  });

  const data = await r.json();
  if (data.status === "ok") location.reload();
  else alert("Error updating status");
}
</script>
</body>
</html>


In [ ]:
%%writefile templates/admin_assigned.html
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8">
  <meta name="viewport" content="width=device-width, initial-scale=1.0">
  <title>CivicCare — Assigned Issues</title>
  <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">
</head>

<body class="admin">

{% include "navbar.html" %}

<main class="container">

  <div class="panel-container">

    <div class="panel-header">
      <div class="panel-title">Assigned Issues</div>
    </div>

    {% if assigned|length == 0 %}
      <div class="no-issues">No assigned issues.</div>
    {% endif %}

    {% for issue in assigned %}
    <div class="issue-card">

      <div class="issue-title">{{ issue.title }}</div>

      <div class="issue-meta">
        Category: {{ issue.category }} |
        ID: {{ issue.id }} |
        Taken Up At: {{ issue.assigned_at or "—" }}
      </div>

      <div class="issue-desc">{{ issue.description }}</div>

      <div class="action-buttons">
        <button class="btn-action btn-resolve"
                onclick="resolveIssue('{{ issue.id }}')">✅ Mark Resolved</button>
      </div>

    </div>
    {% endfor %}

  </div>

</main>

<script>
async function resolveIssue(id) {
  const r = await fetch("/api/admin_resolve_issue", {
    method: "POST",
    headers: {"Content-Type": "application/json"},
    body: JSON.stringify({id})
  });

  const data = await r.json();
  if (data.status === "ok") location.reload();
  else alert("Error updating status");
}
</script>

</body>
</html>


In [ ]:
%%writefile templates/admin_resolved.html
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8">
  <meta name="viewport" content="width=device-width, initial-scale=1.0">
  <title>CivicCare — Resolved Issues</title>
  <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">
</head>

<body class="admin">

{% include "navbar.html" %}

<main class="container">

  <div class="panel-container">

    <div class="panel-header">
      <div class="panel-title">Resolved Issues</div>
    </div>

    {% if resolved|length == 0 %}
      <div class="no-issues">No resolved issues.</div>
    {% endif %}

    {% for issue in resolved %}
    <div class="issue-card">

      <div class="issue-title">{{ issue.title }}</div>

      <div class="issue-meta">
        Category: {{ issue.category }} |
        ID: {{ issue.id }} |
        Resolved At: {{ issue.resolved_at or "—" }}
      </div>

      <div class="issue-desc">{{ issue.description }}</div>

    </div>
    {% endfor %}

  </div>

</main>
</body>
</html>


In [ ]:
%%writefile templates/user_pending.html
<!doctype html>
<html lang="en">
<head>
  <meta charset="utf-8"/>
  <meta name="viewport" content="width=device-width,initial-scale=1"/>
  <title>Pending Issues — CivicCare</title>
  <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">
</head>
<body class="user">
  {% include 'navbar.html' %}

  <main class="container">
    <div class="page-header">
      <h2 class="page-title">Pending Issues</h2>
      <p class="page-subtitle">These reports are submitted and waiting for municipal action.</p>
    </div>

    <div class="panel-container">
      {% if pending|length == 0 %}
        <div class="no-issues">No pending issues right now.</div>
      {% endif %}

      {% for issue in pending %}
        <div class="issue-card">
          <div class="issue-title">{{ issue.title }}</div>
          <div class="issue-meta">
            Category: {{ issue.category }} |
            ID: {{ issue.id }} |
            Reported: {{ issue.created_at }}
          </div>
          <div class="issue-desc">{{ issue.description }}</div>
          <div style="margin-top:12px">
            <a class="view-link" href="{{ url_for('issue_page', issue_id=issue.id) }}">View Details →</a>
          </div>
        </div>
      {% endfor %}
    </div>
  </main>
</body>
</html>


In [ ]:
%%writefile templates/user_assigned.html
<!doctype html>
<html lang="en">
<head>
  <meta charset="utf-8"/>
  <meta name="viewport" content="width=device-width,initial-scale=1"/>
  <title>Assigned Issues — CivicCare</title>
  <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">
</head>
<body class="user">
  {% include 'navbar.html' %}

  <main class="container">
    <div class="page-header">
      <h2 class="page-title">Assigned Issues</h2>
      <p class="page-subtitle">These reports are currently being handled by municipal teams.</p>
    </div>

    <div class="panel-container">
      {% if assigned|length == 0 %}
        <div class="no-issues">No issues are assigned yet.</div>
      {% endif %}

      {% for issue in assigned %}
        <div class="issue-card">
          <div class="issue-title">{{ issue.title }}</div>
          <div class="issue-meta">
            Category: {{ issue.category }} |
            ID: {{ issue.id }} |
            Assigned At: {{ issue.assigned_at or 'N/A' }}
          </div>
          <div class="issue-desc">{{ issue.description }}</div>
          <div style="margin-top:12px">
            <a class="view-link" href="{{ url_for('issue_page', issue_id=issue.id) }}">View Details →</a>
          </div>
        </div>
      {% endfor %}
    </div>
  </main>
</body>
</html>


In [ ]:
%%writefile templates/user_resolved.html
<!doctype html>
<html lang="en">
<head>
  <meta charset="utf-8"/>
  <meta name="viewport" content="width=device-width,initial-scale=1"/>
  <title>Resolved Issues — CivicCare</title>
  <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">
</head>
<body class="user">
  {% include 'navbar.html' %}

  <main class="container">
    <div class="page-header">
      <h2 class="page-title">Resolved Issues</h2>
      <p class="page-subtitle">Issues that have been marked resolved by municipal staff.</p>
    </div>

    <div class="panel-container">
      {% if resolved|length == 0 %}
        <div class="no-issues">No resolved issues yet.</div>
      {% endif %}

      {% for issue in resolved %}
        <div class="issue-card">
          <div class="issue-title">{{ issue.title }}</div>
          <div class="issue-meta">
            Category: {{ issue.category }} |
            ID: {{ issue.id }} |
            Resolved At: {{ issue.resolved_at or 'N/A' }}
          </div>
          <div class="issue-desc">{{ issue.description }}</div>
          <div style="margin-top:12px">
            <a class="view-link" href="{{ url_for('issue_page', issue_id=issue.id) }}">View Details →</a>
          </div>
        </div>
      {% endfor %}
    </div>
  </main>
</body>
</html>


In [ ]:
%%writefile admin.json
{
  "username": "civicadmin",
  "password": "city123"
}


In [ ]:
%%writefile templates/public_list.html
<!doctype html>
<html lang="en">
<head>
  <meta charset="utf-8"/>
  <meta name="viewport" content="width=device-width,initial-scale=1"/>
  <title>{{ title }} — CivicCare</title>
  <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">
</head>

<body class="user">

  {% include 'navbar.html' %}

  <main class="container">

    <div class="panel-container">

      <div class="panel-header">
        <div class="panel-title">{{ title }}</div>
      </div>

      {% if issues|length == 0 %}
      <div class="no-issues">No issues found.</div>
      {% endif %}

      {% for issue in issues %}
      <div class="issue-card">
        <div class="issue-title">{{ issue.title }}</div>

        <div class="issue-meta">
          Category: {{ issue.category }} |
          ID: {{ issue.id }} |
          Reported: {{ issue.created_at }}
        </div>

        <div class="issue-desc">{{ issue.description }}</div>

        <div style="margin-top:12px">
          <a class="view-link" href="{{ url_for('issue_page', issue_id=issue.id) }}">
            View Details →
          </a>
        </div>
      </div>
      {% endfor %}

    </div>

  </main>
</body>
</html>


## Step 14 — Issue Detail Template

Displays:
• Title  
• Category  
• Status  
• Description  
• Original + Annotated images  
• Map link (OpenStreetMap)  


In [ ]:
%%writefile templates/issue.html
<!doctype html>
<html lang="en">
<head>
  <meta charset="utf-8"/>
  <meta name="viewport" content="width=device-width, initial-scale=1"/>
  <title>Issue Details — CivicCare</title>
  <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">
</head>
<body class="{% if session.get('admin') %}admin{% else %}user{% endif %}">

<!-- SHARED NAVBAR -->
{% include 'navbar.html' %}

<main class="container">

  <div class="issue-detail-card">

    <div class="issue-header">
      <h1 class="issue-title">{{ issue.title }}</h1>
      <div class="issue-status-badge {{ issue.status or 'pending' }}">
        {{ (issue.status or 'pending').upper() }}
      </div>
    </div>

    <div class="issue-meta-grid">

      <!-- CATEGORY -->
      <div class="meta-item">
        <span class="meta-icon">📁</span>
        <div>
          <div class="meta-label">Category</div>

          <div class="meta-value">
            {% set cat = (issue.category or '').lower() %}

            {% if "damaged concrete" in cat %}
              Damaged Concrete Structures
            {% elif "electrical" in cat %}
              Damaged Electrical Poles
            {% elif "roadsign" in cat or "sign" in cat %}
              Damaged Road Signs
            {% elif "fallen" in cat or "trees" in cat %}
              Fallen Trees
            {% elif "garbage" in cat %}
              Garbage
            {% elif "graff" in cat %}
              Graffiti
            {% elif "pothole" in cat or "crack" in cat %}
              Potholes & Road Cracks
            {% else %}
              Other
            {% endif %}
          </div>

        </div>
      </div>

      <!-- TIME -->
      <div class="meta-item">
        <span class="meta-icon">🕐</span>
        <div>
          <div class="meta-label">Reported At</div>
          <div class="meta-value">{{ issue.created_at or '—' }}</div>
        </div>
      </div>

      <!-- ID -->
      <div class="meta-item">
        <span class="meta-icon">🔑</span>
        <div>
          <div class="meta-label">Issue ID</div>
          <div class="meta-value" style="font-family:monospace;font-size:12px">
            {{ issue.id[:12] }}...
          </div>
        </div>
      </div>
    </div>

    {% if issue.lat and issue.lon %}
    <div class="location-section">
      <h3>📍 Location</h3>

      <div class="location-coords">
        <span><strong>Latitude:</strong> {{ issue.lat }}</span>
        <span><strong>Longitude:</strong> {{ issue.lon }}</span>
      </div>

      <a
        href="https://www.openstreetmap.org/?mlat={{ issue.lat }}&mlon={{ issue.lon }}#map=18/{{ issue.lat }}/{{ issue.lon }}"
        target="_blank"
        class="map-link">
        🗺️ Open in OpenStreetMap
      </a>
    </div>
    {% endif %}

    <div class="description-section">
      <h3>📝 Description</h3>
      <p class="issue-description">{{ issue.description }}</p>
    </div>

    <div class="image-section">
      <h3>📷 Images</h3>

      <div class="image-grid">

        {% if issue.image %}
        <div class="image-block">
          <h4>Original Image</h4>
          <img src="{{ issue.image }}" class="issue-detail-image">
        </div>
        {% endif %}

        {% if issue.annotated %}
        <div class="image-block">
          <h4>AI Annotated Image</h4>
          <img src="{{ issue.annotated }}" class="issue-detail-image">
        </div>
        {% endif %}

        {% if not issue.image and not issue.annotated %}
        <p>No images available.</p>
        {% endif %}

      </div>
    </div>

    <!-- ACTION BUTTONS -->
    <div class="action-buttons">

      {% if session.get('admin') %}
      <a class="submit-button" href="{{ url_for('admin') }}">
        ← Back to Dashboard
      </a>
      {% else %}
      <a class="submit-button" href="{{ url_for('home') }}">
        ← Back to Home
      </a>
      {% endif %}

      <a class="submit-button ghost" href="{{ url_for('report') }}">
        📢 Report New Issue
      </a>

    </div>

  </div>

</main>

<footer class="footer">
  <div>This is a detailed view of the reported civic issue.</div>
  <div style="margin-top:6px;font-size:12px;opacity:0.7">
    Secure endpoints and sanitize inputs before production deployment.
  </div>
</footer>

</body>
</html>



## Step 15 — Add global CSS stylesheet

Contains styles for:
• Layout  
• Components  
• Animations  
• Forms  
• Map UI  
• Admin dashboard  
• User view  


In [ ]:
# ===============================
# 9️⃣ CSS Stylesheet
# ===============================
%%writefile static/style.css
:root {
  --bg: #061026;
  --card: #0e1724;
  --glass: rgba(255,255,255,0.04);
  --text: #ffffff;
  --muted: #c7d2e8;
  --primary: #6366f1;
  --primary-dark: #4f46e5;
  --accent: #10b981;
  --accent-dark: #0a8c62;
  --warning: #f59e0b;
  --error: #ef4444;
  --shadow: 0 18px 40px rgba(0,0,0,0.6);
}

* {
  box-sizing: border-box;
  margin: 0;
  padding: 0;
}

html, body {
  height: 100%;
  scroll-behavior: smooth;
}

body {
  margin: 0;
  background: linear-gradient(135deg, #041025, #0b1730);
  color: var(--text);
  font-family: Inter, ui-sans-serif, system-ui, -apple-system, "Segoe UI", Roboto, "Helvetica Neue", Arial, sans-serif;
  -webkit-font-smoothing: antialiased;
  -moz-osx-font-smoothing: grayscale;
  line-height: 1.6;
}

/* Particles background */
.particles {
  position: fixed;
  inset: 0;
  pointer-events: none;
  z-index: 0;
  background:
    radial-gradient(circle at 10% 15%, rgba(99,102,241,0.07), transparent 20%),
    radial-gradient(circle at 85% 80%, rgba(16,185,129,0.04), transparent 25%);
  filter: blur(10px);
}

/* Topbar */
.topbar {
  position: sticky;
  top: 0;
  z-index: 50;
  display: flex;
  align-items: center;
  justify-content: space-between;
  padding: 14px 26px;
  background: linear-gradient(180deg, rgba(255,255,255,0.02), rgba(255,255,255,0.01));
  border-bottom: 1px solid rgba(255,255,255,0.03);
  backdrop-filter: blur(8px);
}

.topbar-left {
  display: flex;
  align-items: center;
  gap: 12px;
}

.logo-icon {
  font-size: 30px;
}

.brand-title {
  font-weight: 700;
  font-size: 18px;
  color: var(--text);
}

.brand-sub {
  font-size: 12px;
  color: var(--muted);
}

.topbar-right a {
  margin-left: 18px;
  color: var(--muted);
  text-decoration: none;
  font-weight: 600;
  transition: color 0.2s;
}

.topbar-right a.active,
.topbar-right a:hover {
  color: var(--text);
}

/* Container */
.container {
  max-width: 1200px;
  margin: 28px auto;
  padding: 18px;
  position: relative;
  z-index: 2;
}

/* Hero Section */
.hero-section {
  display: flex;
  gap: 40px;
  padding: 48px;
  border-radius: 14px;
  background: var(--card);
  box-shadow: var(--shadow);
  align-items: center;
  margin-bottom: 32px;
}

.hero-left {
  flex: 1;
}

.hero-right {
  width: 320px;
  display: flex;
  justify-content: center;
  align-items: center;
}

.main-title {
  font-size: 48px;
  margin: 0;
  font-weight: 800;
  background: linear-gradient(90deg, var(--primary), var(--accent));
  -webkit-background-clip: text;
  -webkit-text-fill-color: transparent;
  background-clip: text;
}

.hero-subtitle {
  margin-top: 8px;
  color: var(--accent);
  font-weight: 600;
  font-size: 18px;
}

.hero-description {
  margin-top: 16px;
  color: var(--muted);
  line-height: 1.7;
  font-size: 15px;
}

/* CTA Buttons */
.cta-row {
  display: flex;
  gap: 12px;
  margin-top: 24px;
  flex-wrap: wrap;
}

.cta-button,
.cta-button-secondary {
  padding: 14px 24px;
  border-radius: 12px;
  font-weight: 700;
  text-decoration: none;
  display: inline-block;
  transition: transform 0.2s, box-shadow 0.2s;
  font-size: 15px;
}

.cta-button {
  background: linear-gradient(90deg, var(--primary), var(--primary-dark));
  color: white;
  box-shadow: 0 8px 24px rgba(79,70,229,0.3);
}

.cta-button:hover {
  transform: translateY(-2px);
  box-shadow: 0 12px 32px rgba(79,70,229,0.4);
}

.cta-button-secondary {
  background: linear-gradient(90deg, var(--accent), var(--accent-dark));
  color: white;
  box-shadow: 0 8px 24px rgba(16,185,129,0.2);
}

.cta-button-secondary:hover {
  transform: translateY(-2px);
  box-shadow: 0 12px 32px rgba(16,185,129,0.3);
}

/* Stats Section */
.stats-section {
  display: grid;
  grid-template-columns: repeat(auto-fit, minmax(140px, 1fr));
  gap: 16px;
  margin-top: 32px;
}

.stat-card {
  padding: 16px;
  border-radius: 12px;
  background: rgba(255,255,255,0.02);
  border: 1px solid rgba(255,255,255,0.05);
  text-align: center;
}

.stat-number {
  font-size: 20px;
  font-weight: 800;
  color: var(--accent);
}

.stat-label {
  color: var(--muted);
  font-size: 13px;
  margin-top: 4px;
}

/* Floating Card */
.floating-card {
  background: linear-gradient(135deg, rgba(99,102,241,0.1), rgba(16,185,129,0.1));
  padding: 32px;
  border-radius: 16px;
  border: 1px solid rgba(255,255,255,0.08);
  text-align: center;
  box-shadow: 0 12px 32px rgba(0,0,0,0.4);
}

.card-icon-large {
  font-size: 64px;
  margin-bottom: 16px;
}

.floating-card h3 {
  margin: 12px 0 8px 0;
  font-size: 22px;
}

.floating-card p {
  color: var(--muted);
  font-size: 14px;
  line-height: 1.6;
}

/* Features Grid */
.features-grid {
  display: grid;
  grid-template-columns: repeat(auto-fit, minmax(280px, 1fr));
  gap: 20px;
  margin-top: 32px;
}

.feature-card {
  padding: 24px;
  border-radius: 12px;
  background: linear-gradient(180deg, rgba(255,255,255,0.02), rgba(255,255,255,0.005));
  border: 1px solid rgba(255,255,255,0.04);
  transition: transform 0.2s, border-color 0.2s;
}

.feature-card:hover {
  transform: translateY(-4px);
  border-color: rgba(99,102,241,0.3);
}

.card-icon {
  font-size: 36px;
  margin-bottom: 12px;
}

.feature-card h3 {
  margin: 8px 0 12px 0;
  font-size: 18px;
  color: var(--text);
}

.feature-card p {
  color: var(--muted);
  font-size: 14px;
  line-height: 1.6;
}

/* Page Header */
.page-header {
  margin-bottom: 32px;
  text-align: center;
}

.page-title {
  font-size: 36px;
  font-weight: 800;
  margin-bottom: 8px;
}

.page-subtitle {
  color: var(--muted);
  font-size: 15px;
}

/* Form Sections */
.main-form {
  background: var(--card);
  padding: 32px;
  border-radius: 14px;
  box-shadow: var(--shadow);
  border: 1px solid rgba(255,255,255,0.04);
  max-width: 800px;
  margin: 0 auto;
}

.form-section {
  margin-bottom: 32px;
  padding-bottom: 32px;
  border-bottom: 1px solid rgba(255,255,255,0.05);
}

.form-section:last-child {
  border-bottom: none;
  margin-bottom: 0;
  padding-bottom: 0;
}

.section-title {
  font-size: 20px;
  font-weight: 700;
  margin-bottom: 8px;
  color: var(--text);
}

.section-desc {
  color: var(--muted);
  font-size: 14px;
  margin-bottom: 20px;
}

/* Upload Area */
.upload-area {
  margin: 20px 0;
  position: relative;
  border: 2px dashed rgba(255,255,255,0.1);
  border-radius: 12px;
  transition: border-color 0.3s;
}

.upload-placeholder {
  display: flex;
  flex-direction: column;
  align-items: center;
  justify-content: center;
  padding: 48px 24px;
  cursor: pointer;
  color: var(--muted);
  transition: background 0.2s;
}

.upload-placeholder:hover {
  background: rgba(255,255,255,0.02);
}

.upload-icon {
  font-size: 48px;
  margin-bottom: 16px;
}

.image-preview {
  width: 100%;
  max-height: 400px;
  object-fit: contain;
  border-radius: 10px;
  padding: 12px;
}

/* Form Inputs */
.input-label {
  display: block;
  margin-top: 18px;
  margin-bottom: 8px;
  color: var(--muted);
  font-weight: 600;
  font-size: 13px;
  text-transform: uppercase;
  letter-spacing: 0.5px;
}

input[type="text"],
input[type="file"],
textarea {
  width: 100%;
  padding: 12px 16px;
  border-radius: 10px;
  border: 1px solid rgba(255,255,255,0.08);
  background: rgba(255,255,255,0.03);
  color: var(--text);
  font-size: 15px;
  font-family: inherit;
  transition: border-color 0.2s, background 0.2s;
}

input:focus,
textarea:focus {
  outline: none;
  border-color: var(--primary);
  background: rgba(255,255,255,0.05);
}

input[readonly] {
  background: rgba(255,255,255,0.01);
  cursor: not-allowed;
  opacity: 0.7;
}

textarea {
  min-height: 120px;
  resize: vertical;
  line-height: 1.6;
}

/* Form Row/Col */
.form-row {
  display: grid;
  grid-template-columns: 1fr 1fr;
  gap: 16px;
  margin-top: 16px;
}

.form-col {
  position: relative;
}

.detect-btn {
  margin-top: 8px;
  padding: 8px 14px;
  border-radius: 8px;
  border: 1px solid rgba(255,255,255,0.1);
  background: rgba(255,255,255,0.05);
  color: var(--text);
  font-size: 13px;
  cursor: pointer;
  font-weight: 600;
  transition: all 0.2s;
}

.detect-btn:hover {
  background: var(--primary);
  border-color: var(--primary);
}

/* Action Buttons */
.action-row {
  display: flex;
  gap: 12px;
  margin-top: 20px;
  flex-wrap: wrap;
  align-items: center;
}

.submit-button {
  padding: 12px 24px;
  border-radius: 10px;
  border: none;
  font-weight: 700;
  cursor: pointer;
  font-size: 15px;
  transition: all 0.2s;
  text-decoration: none;
  display: inline-block;
  text-align: center;
}

.submit-button.primary {
  background: linear-gradient(90deg, var(--primary), var(--primary-dark));
  color: white;
  box-shadow: 0 4px 12px rgba(99,102,241,0.3);
}

.submit-button.primary:hover:not(:disabled) {
  transform: translateY(-2px);
  box-shadow: 0 6px 16px rgba(99,102,241,0.4);
}

.submit-button.success {
  background: linear-gradient(90deg, var(--accent), var(--accent-dark));
  color: white;
  box-shadow: 0 4px 12px rgba(16,185,129,0.3);
}

.submit-button.success:hover:not(:disabled) {
  transform: translateY(-2px);
  box-shadow: 0 6px 16px rgba(16,185,129,0.4);
}

.submit-button.ghost {
  background: transparent;
  border: 1px solid rgba(255,255,255,0.1);
  color: var(--text);
}

.submit-button.ghost:hover {
  background: rgba(255,255,255,0.05);
  border-color: rgba(255,255,255,0.2);
}

.submit-button:disabled {
  opacity: 0.5;
  cursor: not-allowed;
}

.view-link {
  color: var(--accent);
  font-weight: 600;
  text-decoration: none;
  font-size: 14px;
  transition: color 0.2s;
}

.view-link:hover {
  color: var(--accent-dark);
  text-decoration: underline;
}

/* AI Badge */
.ai-badge {
  display: inline-block;
  padding: 8px 16px;
  border-radius: 20px;
  font-weight: 700;
  font-size: 13px;
  margin-bottom: 16px;
  text-transform: uppercase;
  letter-spacing: 0.5px;
}

.ai-badge.pothole {
  background: rgba(239,68,68,0.2);
  color: #fca5a5;
  border: 1px solid rgba(239,68,68,0.3);
}

.ai-badge.garbage {
  background: rgba(245,158,11,0.2);
  color: #fcd34d;
  border: 1px solid rgba(245,158,11,0.3);
}

.ai-badge.streetlight {
  background: rgba(234,179,8,0.2);
  color: #fde047;
  border: 1px solid rgba(234,179,8,0.3);
}

.ai-badge.water-leakage {
  background: rgba(59,130,246,0.2);
  color: #93c5fd;
  border: 1px solid rgba(59,130,246,0.3);
}

.ai-badge.blocked-drain {
  background: rgba(6,182,212,0.2);
  color: #67e8f9;
  border: 1px solid rgba(6,182,212,0.3);
}

.ai-badge.illegal-parking {
  background: rgba(139,92,246,0.2);
  color: #c4b5fd;
  border: 1px solid rgba(139,92,246,0.3);
}

.ai-badge.other {
  background: rgba(107,114,128,0.2);
  color: #d1d5db;
  border: 1px solid rgba(107,114,128,0.3);
}

/* Confidence Display */
.confidence-display {
  margin-top: 16px;
  padding: 12px;
  background: rgba(255,255,255,0.02);
  border-radius: 8px;
  border: 1px solid rgba(255,255,255,0.05);
  font-size: 13px;
  color: var(--muted);
  display: none;
}

/* Status Messages */
.status-message {
  margin-top: 20px;
  padding: 14px 18px;
  border-radius: 10px;
  font-weight: 600;
  font-size: 14px;
  display: none;
}

.status-message.info {
  background: rgba(59,130,246,0.1);
  color: #93c5fd;
  border: 1px solid rgba(59,130,246,0.2);
  display: block;
}

.status-message.success {
  background: rgba(16,185,129,0.1);
  color: #6ee7b7;
  border: 1px solid rgba(16,185,129,0.2);
  display: block;
}

.status-message.error {
  background: rgba(239,68,68,0.1);
  color: #fca5a5;
  border: 1px solid rgba(239,68,68,0.2);
  display: block;
}

.status-message.warning {
  background: rgba(245,158,11,0.1);
  color: #fcd34d;
  border: 1px solid rgba(245,158,11,0.2);
  display: block;
}

/* Stats Row (Admin) */
.stats-row {
  display: grid;
  grid-template-columns: repeat(auto-fit, minmax(200px, 1fr));
  gap: 20px;
  margin-bottom: 32px;
}

.stat-box {
  background: var(--card);
  padding: 24px;
  border-radius: 12px;
  border: 1px solid rgba(255,255,255,0.05);
  display: flex;
  align-items: center;
  gap: 16px;
  box-shadow: 0 4px 12px rgba(0,0,0,0.3);
}

.stat-icon {
  font-size: 36px;
}

.stat-value {
  font-size: 32px;
  font-weight: 800;
  color: var(--primary);
  line-height: 1;
}

.stat-label {
  color: var(--muted);
  font-size: 13px;
  margin-top: 4px;
}

/* Filter Bar */
.filter-bar {
  display: flex;
  gap: 10px;
  padding: 16px;
  border-radius: 12px;
  background: var(--card);
  margin-bottom: 20px;
  overflow-x: auto;
  border: 1px solid rgba(255,255,255,0.04);
}

.filter-btn {
  padding: 10px 18px;
  border-radius: 8px;
  border: none;
  background: rgba(255,255,255,0.04);
  color: var(--text);
  cursor: pointer;
  font-weight: 600;
  font-size: 14px;
  white-space: nowrap;
  transition: all 0.2s;
}

.filter-btn:hover {
  background: rgba(255,255,255,0.08);
}

.filter-btn.active {
  background: var(--primary);
  color: white;
  box-shadow: 0 2px 8px rgba(99,102,241,0.3);
}

/* Map Container */
.map-container {
  height: 600px;
  border-radius: 12px;
  overflow: hidden;
  box-shadow: var(--shadow);
  border: 1px solid rgba(255,255,255,0.05);
}

/* Map Popup Styles */
.popup-wrap {
  max-width: 280px;
  padding: 4px;
}

.popup-wrap h3 {
  margin: 0 0 8px 0;
  font-size: 16px;
  color: #1f2937;
}

.popup-category {
  display: inline-block;
  padding: 4px 10px;
  border-radius: 12px;
  font-size: 11px;
  font-weight: 700;
  margin-bottom: 10px;
  color: white;
}

.popup-wrap p {
  margin: 8px 0;
  font-size: 13px;
  line-height: 1.5;
  color: #4b5563;
}

.popup-meta {
  font-size: 12px;
  color: #6b7280;
  margin: 4px 0;
}

.popup-img {
  width: 100%;
  border-radius: 8px;
  margin-top: 12px;
  margin-bottom: 8px;
}

.popup-wrap .view-link {
  display: inline-block;
  margin-top: 10px;
  font-size: 13px;
}

/* Issue Detail Page */
.issue-detail-card {
  background: var(--card);
  padding: 40px;
  border-radius: 14px;
  box-shadow: var(--shadow);
  border: 1px solid rgba(255,255,255,0.04);
  max-width: 900px;
  margin: 0 auto;
}

.issue-header {
  display: flex;
  justify-content: space-between;
  align-items: flex-start;
  margin-bottom: 32px;
  gap: 20px;
}

.issue-title {
  font-size: 32px;
  font-weight: 800;
  color: var(--text);
  flex: 1;
  line-height: 1.3;
}

.issue-status-badge {
  padding: 8px 16px;
  border-radius: 20px;
  font-size: 12px;
  font-weight: 700;
  letter-spacing: 1px;
}

.issue-status-badge.pending {
  background: rgba(245,158,11,0.2);
  color: #fcd34d;
  border: 1px solid rgba(245,158,11,0.3);
}

.issue-status-badge.resolved {
  background: rgba(16,185,129,0.2);
  color: #6ee7b7;
  border: 1px solid rgba(16,185,129,0.3);
}

.issue-meta-grid {
  display: grid;
  grid-template-columns: repeat(auto-fit, minmax(200px, 1fr));
  gap: 20px;
  margin-bottom: 32px;
}

.meta-item {
  display: flex;
  align-items: center;
  gap: 12px;
  padding: 16px;
  background: rgba(255,255,255,0.02);
  border-radius: 10px;
  border: 1px solid rgba(255,255,255,0.05);
}

.meta-icon {
  font-size: 24px;
}

.meta-label {
  font-size: 12px;
  color: var(--muted);
  text-transform: uppercase;
  letter-spacing: 0.5px;
  font-weight: 600;
}

.meta-value {
  font-size: 15px;
  color: var(--text);
  font-weight: 600;
  margin-top: 2px;
}

.location-section,
.description-section,
.image-section {
  margin-bottom: 32px;
}

.location-section h3,
.description-section h3,
.image-section h3 {
  font-size: 18px;
  margin-bottom: 16px;
  color: var(--text);
}

.location-coords {
  display: flex;
  gap: 24px;
  margin-bottom: 12px;
  font-size: 14px;
  color: var(--muted);
}

.map-link {
  display: inline-block;
  padding: 10px 18px;
  background: var(--primary);
  color: white;
  text-decoration: none;
  border-radius: 8px;
  font-weight: 600;
  font-size: 14px;
  transition: all 0.2s;
}

.map-link:hover {
  background: var(--primary-dark);
  transform: translateY(-2px);
}

.issue-description {
  color: var(--muted);
  line-height: 1.7;
  font-size: 15px;
}

.issue-detail-image {
  width: 100%;
  max-height: 500px;
  object-fit: contain;
  border-radius: 12px;
  border: 1px solid rgba(255,255,255,0.05);
}

.action-buttons {
  display: flex;
  gap: 12px;
  margin-top: 32px;
  flex-wrap: wrap;
}

/* Footer */
.footer {
  text-align: center;
  padding: 24px;
  margin-top: 48px;
  color: var(--muted);
  font-size: 14px;
  border-top: 1px solid rgba(255,255,255,0.03);
}

/* Responsive */
@media (max-width: 768px) {
  .hero-section {
    flex-direction: column;
    padding: 32px 24px;
  }

  .hero-right {
    width: 100%;
  }

  .main-title {
    font-size: 36px;
  }

  .features-grid {
    grid-template-columns: 1fr;
  }

  .map-container {
    height: 400px;
  }

  .form-row {
    grid-template-columns: 1fr;
  }

  .stats-row {
    grid-template-columns: 1fr;
  }

  .filter-bar {
    flex-wrap: wrap;
  }

  .issue-detail-card {
    padding: 24px;
  }

  .issue-title {
    font-size: 24px;
  }

  .issue-header {
    flex-direction: column;
  }

  .location-coords {
    flex-direction: column;
    gap: 8px;
  }
}

/* Custom Marker */
.custom-marker {
  background: transparent;
  border: none;
}

/* Scrollbar */
::-webkit-scrollbar {
  width: 8px;
  height: 8px;
}

::-webkit-scrollbar-track {
  background: rgba(255,255,255,0.02);
}

::-webkit-scrollbar-thumb {
  background: rgba(255,255,255,0.1);
  border-radius: 4px;
}

::-webkit-scrollbar-thumb:hover {
  background: rgba(255,255,255,0.15);
}

/* ============================================
   Issue Detail — Image Grid (Original + Annotated)
   ============================================ */

.image-grid {
  display: grid;
  grid-template-columns: repeat(auto-fit, minmax(280px, 1fr));
  gap: 24px;
  margin-top: 12px;
}

.image-block {
  background: rgba(255,255,255,0.03);
  border: 1px solid rgba(255,255,255,0.05);
  padding: 16px;
  border-radius: 12px;
  box-shadow: 0 6px 20px rgba(0,0,0,0.3);
}

.image-block h4 {
  font-size: 15px;
  color: var(--muted);
  margin-bottom: 12px;
  font-weight: 700;
  text-transform: uppercase;
  letter-spacing: 0.5px;
}

.image-block img {
  width: 100%;
  max-height: 380px;
  object-fit: contain;
  border-radius: 10px;
  border: 1px solid rgba(255,255,255,0.08);
  background: rgba(0,0,0,0.2);
  padding: 8px;
}

/* Responsive image grid */
@media (max-width: 600px) {
  .image-grid {
    grid-template-columns: 1fr;
  }
}

/* ============================================
   NAVBAR FIXES — Shared navbar.html support
   ============================================ */

/* Ensure navbar items align nicely */
.topbar-right {
  display: flex;
  align-items: center;
}

.topbar-right a {
  padding: 6px 10px;
  border-radius: 6px;
}

.topbar-right a.active {
  background: rgba(255,255,255,0.08);
  color: white;
}

/* Show Admin-only menu only when admin logged in */
.admin-only {
  display: none;
}

body.admin .admin-only {
  display: inline-block;
}

/* Hide user-only items when admin logged in */
body.admin .user-only {
  display: none;
}

/* For users: hide admin-only links */
body.user .admin-only {
  display: none;
}


/* ============================================
   POPUP FIX — text is dark and unreadable
   ============================================ */

.popup-wrap h3 {
  color: var(--text) !important;
}

.popup-wrap p {
  color: var(--muted) !important;
}

.popup-meta {
  color: var(--muted) !important;
}

.leaflet-popup-content-wrapper {
  background: var(--card) !important;
  color: var(--text) !important;
  border: 1px solid rgba(255,255,255,0.08);
  backdrop-filter: blur(4px);
}

.leaflet-popup-tip {
  background: var(--card) !important;
}


/* ============================================
   ADMIN PANELS (Pending / Assigned / Resolved)
   ============================================ */

.panel-container {
  background: var(--card);
  border: 1px solid rgba(255,255,255,0.04);
  box-shadow: var(--shadow);
  border-radius: 14px;
  padding: 30px;
}

.issue-card {
  background: rgba(255,255,255,0.03);
  border: 1px solid rgba(255,255,255,0.05);
  padding: 20px;
  border-radius: 12px;
  margin-bottom: 16px;
}

.issue-card:hover {
  border-color: rgba(255,255,255,0.15);
}

.issue-title {
  color: var(--text);
  font-size: 20px;
  font-weight: 700;
}

.issue-meta {
  color: var(--muted);
  font-size: 13px;
}

.btn-takeup,
.btn-resolve {
  padding: 10px 16px;
  background: var(--primary);
  color: white;
  border-radius: 8px;
  border: none;
  font-weight: 700;
  cursor: pointer;
  transition: 0.2s;
}

.btn-takeup:hover,
.btn-resolve:hover {
  background: var(--primary-dark);
  transform: translateY(-2px);
}


/* ============================================
   ISSUE STATUS BADGES (Pending / Assigned / Resolved)
   ============================================ */

.issue-status-badge.pending {
  background: rgba(245,158,11,0.25);
  color: #fcd34d;
}

.issue-status-badge.assigned {
  background: rgba(59,130,246,0.25);
  color: #93c5fd;
  border: 1px solid rgba(59,130,246,0.4);
}

.issue-status-badge.resolved {
  background: rgba(16,185,129,0.25);
  color: #6ee7b7;
}


/* ============================================
   NAVBAR: Highlight current page automatically
   ============================================ */

nav a.active {
  background: rgba(255,255,255,0.08);
  border-radius: 6px;
}


/* ============================================
   FIX: Make map marker popups consistent
   ============================================ */

.custom-marker div {
  box-shadow: 0 2px 6px rgba(0,0,0,0.5);
}


/* ============================================
   FIX: Improve mobile navbar spacing
   ============================================ */

@media (max-width: 768px) {
  .topbar-right {
    flex-wrap: wrap;
    gap: 8px;
  }
}



## Step 16 — Create requirements.txt (optional)

Useful for exporting to other environments.


In [ ]:
%%writefile requirements.txt
flask==2.2.5
pyngrok==5.1.0

# Hugging Face Core
transformers[torch]==4.34.0
accelerate==0.22.0
huggingface-hub==0.18.1

# Quantization
bitsandbytes==0.39.0

# Tokenizers & NLP
sentencepiece==0.1.98
protobuf==4.23.4

# Audio Processing
soundfile==0.12.1

# Image Handling
pillow==10.0.0

# Datasets (optional but recommended)
datasets==2.14.5

## Step 17 — Start Flask server + expose via ngrok

Workflow:
1. Kill any previous Flask/ngrok instances  
2. Start Flask in background  
3. Start ngrok tunnel on port 8000  
4. Print public URL  

Use this URL on mobile or share for demo.


In [ ]:
!pkill -f flask || echo "No flask running"
!pkill -f ngrok || echo "No ngrok running"

In [ ]:
!lsof -i :8000

In [ ]:
!kill -9 1013

In [ ]:
!nohup python app.py > flask.log 2>&1 &


In [ ]:
from pyngrok import ngrok, conf
conf.get_default().auth_token = "🔑 replace with your ngrok token"  # 🔑 replace with your ngrok token

public_url = ngrok.connect(8000)
print("🌍 Public URL:", public_url)

!sleep 3 && tail -n 30 flask.log


## Step 18 — Debug log preview

View last 20–50 lines of flask.log to check:
• YOLO model loaded?
• LLM loaded?
• API errors?


In [ ]:
!tail -n 50 flask.log

In [ ]:
Your CivicCare Web App is now LIVE!
Use the public ngrok URL to submit reports from mobile or desktop.

Admin login:
username: civicadmin
password: city123

Next:
- Add mobile app UI
- Connect to databases (MongoDB / Firebase)
- Add worker assignment system
- Build analytics dashboard
